# Analysis of Leader Election and Rotation Strategies in BFT Protocols

__Rotation Trigger:__ Is it reactive (failure-driven) or proactive (part of the normal protocol)?

__View Change Cost:__ How expensive is it to replace a leader in terms of performance (communication complexity, downtime)?

- __PBFT:__ Reactive, High Cost (Quadratic). This is the classic, baseline approach.

- __Zyzzyva:__ Reactive, High Cost. It's innovative in its optimistic path but falls back to a PBFT-style view change. So, its leader change *mechanism* is not fundamentally different from PBFT's.

- __SBFT:__ Reactive, Medium Cost (Linear). This is a direct optimization of PBFT's view change. It's different, but it's an incremental improvement on the same reactive model.

- __HotStuff:__ Proactive, Low Cost. This is a fundamentally different philosophy. Leader rotation is part of the normal operation, not an exception. The cost is minimal.

- __Prime & CheapBFT:__ Reactive, High Cost. Similar to PBFT.

Among these protocols, the most distinct categories of leader change are:

- __PBFT:__ Represents the __classic failure-driven__ approach. The leader is fixed until it's suspected to be faulty, which then triggers a very expensive, high-coordination view change. It's the baseline against which others are often measured.

- __SBFT:__ Represents the __optimized failure-driven__ approach. It keeps the same reactive philosophy as PBFT (only change the leader on failure) but makes the view change process significantly more efficient (linear communication instead of quadratic). It's a direct improvement on the *cost* of leader change, making it a distinct category.

- __HotStuff:__ Represents a completely different philosophy: __proactive rotation__. Instead of reacting to failure, the leader rotates as a normal part of the protocol's operation, guided by a "pacemaker." This makes leader failure a non-issue, as the system is designed to move to the next leader in sequence anyway. The cost of a "change" is virtually zero because it's built into the protocol's happy path.




## 1. PBFT (Practical Byzantine Fault Tolerance)

**Leader Strategy: Fixed Leader with View Change on Failure**

- **Leader Selection:** In PBFT, the leader (called the 'primary') is fixed for a given view. The primary is determined by the formula `p = v mod |R|`, where `v` is the view number and `|R|` is the number of replicas. This results in a round-robin selection of the leader as the view number increments.
- **Rotation Strategy:** The leader only changes when it is suspected to be faulty. If replicas do not receive a valid request from the primary within a certain timeout, they trigger a 'view change' protocol. This is a reactive, failure-driven approach.
- **How it Works:** The view change mechanism is complex and involves multiple phases where replicas agree to move to a new view with a new primary. This process introduces a period of liveness failure (the system doesn't make progress) until a new leader is established.

**Expected Performance Implications:**
- **Steady State:** High performance when the leader is stable and honest.
- **On Failure:** Significant latency spike and a drop in throughput during a view change. The cost of a view change is a major performance drawback of PBFT.
- **Metrics to Watch:** To observe this, one would need to design an experiment that forces leader failure. Key metrics would be the duration of the service outage and the latency of requests immediately following the failure.


## 2. SBFT (Simplified Byzantine Fault Tolerance)

**Leader Strategy: Fixed Leader with Linear View Change**

- **Leader Selection:** SBFT also uses a fixed leader per view.
- **Rotation Strategy:** SBFT's main contribution is a more efficient, linear view change protocol. Unlike PBFT's quadratic communication complexity during a view change, SBFT uses a collector-aggregator pattern to reduce the number of messages required.

**Expected Performance Implications:**
- **Steady State:** Performance is similar to PBFT.
- **On Failure:** The key improvement is a much faster recovery time after a leader failure. The latency spike during a view change should be significantly smaller and shorter than in PBFT.
- **Metrics to Watch:** An experiment with leader failures would show SBFT having a much higher throughput and lower latency *during* the period of failure and recovery compared to PBFT.

## 3. HotStuff

**Leader Strategy: Proactive Leader Rotation**

- **Leader Selection:** HotStuff introduces the concept of a 'pacemaker' which dictates when leader rotation should happen. The leader rotates after each successfully committed block (or after a set number of blocks).
- **Rotation Strategy:** This is a proactive, not reactive, rotation. The leadership changes as part of the normal operation of the protocol. This is often called a 'rotating leader' model.

**Expected Performance Implications:**
- **Steady State:** Because the leader is always changing, the performance is more evenly distributed across all replicas. There is no single replica that is a long-term bottleneck. This can improve fairness and decentralization.
- **On Failure:** If a leader fails, the system simply waits for the pacemaker to nominate the next leader in the sequence. There is no complex, multi-phase view change protocol. This makes recovery from leader failure extremely fast and cheap.
- **Metrics to Watch:** HotStuff should exhibit very consistent performance. Even with leader failures, the impact on throughput and latency should be minimal compared to the dramatic dips seen in PBFT-style protocols.

config for sbft and pbft
```
# - state: wait_view_change
#   update: view
#   condition:
#     type: timeout
#     mode: sequence
```